In [1]:
import pandas as pd
from datasets import Dataset
TRAIN_PATH = "../dataset/us_train_data_final_OFFICIAL.jsonl"
train_df = pd.read_json(TRAIN_PATH, lines=True)
train_df.dropna(subset=["text", "summary"], inplace=True)
train_df.head(5)

train = Dataset.from_pandas(train_df)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_df.head(5)

,bill_id,text,summary,title,text_len,sum_len
0,107_hr2256,SECTION 1. SHORT TITLE.\n\n This Act may be...,Border Hospital Survival and Illegal Immigrant...,To amend the Public Health Service Act to esta...,6100,527
1,111_hr4710,SECTION 1. SHORT TITLE.\n\n This Act may be...,Farm to School Improvements Act of 2010 - Amen...,To amend the Richard B. Russell National Schoo...,8628,1161
2,107_s409,SECTION 1. SHORT TITLE.\n\n This Act may be...,Persian Gulf War Illness Compensation Act of 2...,"A bill to amend title 38, United States Code, ...",5567,694
3,109_s2759,SECTION 1. SHORT TITLE.\n\n This Act may be...,Medicare Part D Outreach and Enrollment Enhanc...,A bill to provide for additional outreach and ...,6361,810
4,107_hr5568,SECTION 1. SHORT TITLE.\n\n This Act may be...,Seniors' Retirement Recovery Act of 2002 - Ame...,To amend the Internal Revenue Code of 1986 to ...,5368,380


In [7]:
train_df.iloc[0]["summary"]

'Border Hospital Survival and Illegal Immigrant Care Act - Amends the Public Health Service Act to direct the Secretary of Health and Human Services to establish a five-year pilot program of health care provider reimbursement for the costs associated with providing emergency medical and ambulance services in Arizona to: (1) illegal aliens who are not detained by any Federal, State, or local law enforcement authority. Or (2) aliens paroled into the United States for less than one year to receive emergency medical treatment.'

In [5]:
from transformers import AutoTokenizer

MODEL_ID = "google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [11]:
MAX_SEQ_LENGTH = 2048
MAX_SUMMARY_LENGTH = 256
def filter_long_examples(
    dataset, tokenizer,
):
    """
    Filter out examples that exceed the maximum prompt length.

    Args:
        dataset: HuggingFace Dataset to filter
        tokenizer: Tokenizer for computing sequence lengths
        max_length (int): Maximum allowed prompt length in tokens

    Returns:
        Dataset: Filtered dataset with only examples within length limit
    """

    def is_short_enough(batch):
        tokenized_texts = tokenizer(
            batch["text"],
            truncation=False,
        )
        tokenized_summaries = tokenizer(
            batch["summary"],
            truncation=False,
        )

        return [len(text_ids) < MAX_SEQ_LENGTH and len(sumary_ids) < MAX_SUMMARY_LENGTH 
                for text_ids, sumary_ids in zip(tokenized_texts["input_ids"], tokenized_summaries["input_ids"])]

    return dataset.filter(is_short_enough, batched=True)

In [12]:
train_filtered = filter_long_examples(train, tokenizer)
len(train_filtered)

Filter: 100%|██████████| 18949/18949 [00:09<00:00, 1992.30 examples/s]


7370

In [14]:

dataset = train_filtered.remove_columns(["sum_len", "text_len"]) 
shuffled_ds = dataset.shuffle(seed=42)

sft_dataset_train = shuffled_ds.select(range(2500))
sft_dataset_dev = shuffled_ds.select(range(2500, 3000))
grpo_dataset_train = shuffled_ds.select(range(3000, 4500))
grpo_dataset_dev = shuffled_ds.select(range(4500, 5000))

sft_dataset_train.to_json("../experiments/v2/dataset/sft_dataset_train.jsonl")
sft_dataset_dev.to_json("../experiments/v2/dataset/sft_dataset_dev.jsonl")
grpo_dataset_train.to_json("../experiments/v2/dataset/rl_dataset_train.jsonl")
grpo_dataset_dev.to_json("../experiments/v2/dataset/rl_dataset_dev.jsonl")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 31.21ba/s]


3973603